In [1]:
#dependencies
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt

In [2]:
def read_images(path,num_images,xdim,ydim):
    images = []
    all_paths = os.listdir(path)
    mini_set = all_paths[:num_images]
    for i in mini_set:
        file = path+"/"+i
        image = cv2.imread(file)
        image = cv2.resize(image,(xdim,ydim))
        images.append(image)

    return images


#function to convert rgb to lab 
def rgb_to_lab(images):
    lab_images = []
    for i in images:
        lab_image= cv2.cvtColor(i, cv2.COLOR_RGB2LAB)
        lab_images.append(lab_image)

    return lab_images

def lab_to_rgb(images):
    rgb_images = []
    for i in images:
        lab_image= cv2.cvtColor(i, cv2.COLOR_LAB2RGB)
        lab_images.append(lab_image)

#function to extract the l channels and ab for training
def extract_channels(lab_images):
    l_channels = []
    a_channels = []
    b_channels = []
    for i in lab_images:
        l,a,b = cv2.split(i)
        l_channels.append(l)
        a_channels.append(a)
        b_channels.append(b)

    return np.array(l_channels), np.array(a_channels), np.array(b_channels)

#function to create train and test data
def create_train_data(l,a,b):
    train_data = []
    for i in l:
        train_data.append(np.array(i,dtype= 'float32'))
    train_labels_a = []
    train_labels_b = []
    for i in a:
        train_labels_a.append(np.array(i.flatten(),dtype='float32'))
    for i in b:
        train_labels_b.append(np.array(i.flatten(),dtype='float32'))
    train_labels = []
    for i,j in zip(train_labels_a,train_labels_b):
        train_labels.append(np.concatenate((i,j),axis = 0))
    
    return train_data, train_labels
        

In [3]:
path = "C:/Users/Arghyadeep/Desktop/image colorization/new process/train2017"  #path needs to be changed
num_images = 50000   #this number can be changed according to number of image data in train folder
#converting all images to 128*128
x_dim = 128
y_dim = 128
images = read_images(path,num_images,x_dim,y_dim) #read images
lab_images = rgb_to_lab(images) #convert rgb to lab space images
l,a,b = extract_channels(lab_images) #separate l, a and b channels
train_data, train_labels = create_train_data(l,a,b)

In [4]:
train_data = np.reshape(train_data,(num_images,x_dim,y_dim,1))
train_data.shape[3]
train_labels = np.reshape(train_labels,(num_images,128*128*2))

In [5]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D as Conv2D, MaxPooling2D
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras import backend as K
from keras import losses
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [6]:
img_cols = 128
img_rows = 128
if K.image_data_format() == 'channels_first':
    train_data = train_data.reshape(train_data.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    train_data = train_data.reshape(train_data.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

model = Sequential()
model.add(Conv2D(16, (5, 5),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, (4, 4),activation='relu',input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (4, 4), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())


#model.add(Conv2D(256, (3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
#model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(train_labels.shape[1], activation='linear'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 124, 124, 16)      416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 59, 59, 16)        4112      
_________________________________________________________________
batch_normalization_1 (Batch (None, 59, 59, 16)        64        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 29, 29, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        8224      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
__________

In [7]:
model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [8]:
batch_size = 50
epochs = 100
model.fit(train_data, train_labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

Epoch 1/100
500/500 [==============================] - 10s 20ms/step - loss: 16068.8980 - acc: 0.0000e+00
Epoch 2/100
500/500 [==============================] - 3s 6ms/step - loss: 16048.0593 - acc: 0.0000e+00
Epoch 3/100
500/500 [==============================] - 3s 5ms/step - loss: 16015.3425 - acc: 0.0000e+00A: 0s - loss: 16022.8945 - acc: 0.0000e+
Epoch 4/100
500/500 [==============================] - 3s 5ms/step - loss: 15971.3172 - acc: 0.0000e+00A: 2s - loss: 15919.3743 - ac
Epoch 5/100
500/500 [==============================] - 3s 5ms/step - loss: 15918.3530 - acc: 0.0000e+00
Epoch 6/100
500/500 [==============================] - 3s 5ms/step - loss: 15855.9120 - acc: 0.0000e+00
Epoch 7/100
500/500 [==============================] - 2s 5ms/step - loss: 15783.6992 - acc: 0.0000e+00
Epoch 8/100
500/500 [==============================] - 3s 6ms/step - loss: 15704.1982 - acc: 0.0000e+00
Epoch 9/100
500/500 [==============================] - 3s 7ms/step - loss: 15618.1313 - acc: 0.00

500/500 [==============================] - 2s 5ms/step - loss: 795.3498 - acc: 0.0000e+00A: 1s - loss: 826.2000 - acc: 0.0
Epoch 78/100
500/500 [==============================] - 2s 5ms/step - loss: 680.1198 - acc: 0.0000e+00
Epoch 79/100
500/500 [==============================] - 3s 5ms/step - loss: 575.3607 - acc: 0.0000e+00
Epoch 80/100
500/500 [==============================] - 2s 4ms/step - loss: 484.7620 - acc: 0.0000e+00
Epoch 81/100
500/500 [==============================] - 2s 5ms/step - loss: 403.0434 - acc: 0.0000e+00
Epoch 82/100
500/500 [==============================] - 2s 4ms/step - loss: 335.7539 - acc: 0.0000e+00
Epoch 83/100
500/500 [==============================] - 2s 4ms/step - loss: 279.0752 - acc: 0.0000e+00
Epoch 84/100
500/500 [==============================] - 2s 5ms/step - loss: 236.3241 - acc: 0.0000e+00
Epoch 85/100
500/500 [==============================] - 2s 4ms/step - loss: 205.9693 - acc: 0.0000e+00
Epoch 86/100
500/500 [==============================]

In [9]:
path = "C:/Users/Arghyadeep/Desktop/image colorization/new process/val2017"
num_images = 100
#converting all images to 128*128
x_dim = 128
y_dim = 128
images = read_images(path,num_images,x_dim,y_dim) #read images
lab_images = rgb_to_lab(images) #convert rgb to lab space images
l,a,b = extract_channels(lab_images) #separate l, a and b channels
train_data, train_labels = create_train_data(l,a,b)

In [21]:
index = 50 # choose an image to run test on
tr = np.reshape(train_data[index],(1,128,128,1)) 
p = model.predict(tr) 

In [22]:
x = list(p[0])
length = len(p[0])
x1 = x[:int(length/2)] #divide predicted a and b channels to equal halves to get a and b channels
x2 = x[int(length/2):]

x1 = np.array(np.reshape(x1,(x_dim,y_dim)),dtype = 'uint8')
x2 = np.array(np.reshape(x2,(x_dim,y_dim)),dtype = 'uint8')
#x1 = np.reshape(x1,(64,64))
#x2 = np.reshape(x2,(64,64))
t = np.array(np.reshape(tr,(x_dim,y_dim)),dtype = 'uint8')
pred = cv2.merge((t,x1,x2)) #merge L channel with a and b channels to find colorized image

In [23]:
np.save("pred.pkl",pred) #this pickle is used to plot the image
np.save("bnw.pkl",t) #this pickle is used to plot the L channel image(grayscale)